# Load Bronze Data to Silver Table - Product  - This notebook is not working yet 

## Overview
Load Product sample data from Bronze lakehouse files into Silver lakehouse table.

## Data Flow
- **Source**: MAAG_LH_Bronze/Files/samples_databricks/Product_Samples_ADB.csv
- **Target**: MAAG_LH_Silver.shared.Product table (or any attached default lakehouse)
- **Process**: Read CSV, validate schema, load to Delta table

---

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum as spark_sum
import os

# Configuration - Using correct Fabric cross-lakehouse path from Fabric portal
WORKSPACE_NAME = "Fabric_MAAG"
SOURCE_LAKEHOUSE_NAME = "MAAG_LH_Bronze"
SOURCE_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{SOURCE_LAKEHOUSE_NAME}.Lakehouse/Files/samples_databricks/Product_Samples_ADB.csv"

TARGET_SCHEMA = "shared"
TARGET_TABLE = "Product"
TARGET_FULL_PATH = f"{TARGET_SCHEMA}.{TARGET_TABLE}"

print(f"🔄 Loading Product data")
print(f"📂 Source: {SOURCE_PATH}")
print(f"🎯 Target: {TARGET_FULL_PATH}")

# Read CSV from Bronze lakehouse
df = spark.read.option("header", "true").option("inferSchema", "true").csv(SOURCE_PATH)

print(f"✅ Data loaded successfully")
print(f"📊 Records: {df.count()}")
print(f"📋 Columns: {df.columns}")

# Display sample data
print(f"\n📖 Sample data:")
df.show(10, truncate=False)

In [ ]:
# Validate and conform to target schema
print(f"🔍 Validating data quality...")

# Required columns from Model_Shared_Data.ipynb Product table
required_columns = [
    "ProductID", "ProductName", "ProductDescription", "BrandName", "ProductNumber", "Color", "ProductModel",
    "ProductCategoryID", "CategoryName", "ListPrice", "StandardCost", "Weight", "WeightUom",
    "ProductStatus", "CreatedDate", "SellStartDate", "SellEndDate", "IsoCurrencyCode", "UpdatedDate", "CreatedBy", "UpdatedBy"
]

# Only add/populate UpdatedBy if missing
from pyspark.sql import functions as F
if "UpdatedBy" not in df.columns:
    df = df.withColumn("UpdatedBy", F.lit("Source_Data_Loader"))
    print("✅ Added UpdatedBy column with value 'Source_Data_Loader'.")

print(f"✅ Schema reference (required_columns) retained for documentation/model awareness.")

missing_columns = [c for c in required_columns if c not in df.columns]
if missing_columns:
    print(f"⚠️ Warning: Missing columns in source data: {missing_columns}")
else:
    print(f"✅ All required columns present in source data.")

print(f"✅ Schema validation complete (no error raised for missing columns).")

# Data quality checks
null_counts = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).collect()[0]
print(f"\n📊 Data Quality Check:")
for col_name in df.columns:
    null_count = null_counts[col_name]
    if null_count > 0:
        print(f"  {col_name}: {null_count} null values")
    else:
        print(f"  {col_name}: ✅ No nulls")

# Show value distributions for ProductID
print(f"\n🎯 ProductID Distribution:")
df.groupBy("ProductID").count().orderBy("ProductID").show()

In [ ]:
# Load data to Silver table
print(f"💾 Loading data to Silver table: {TARGET_FULL_PATH}")

# --- Align DataFrame columns to match target table schema (case-sensitive) ---
target_columns = [
    "ProductID", "ProductName", "ProductDescription", "BrandName", "ProductNumber", "Color", "ProductModel",
    "ProductCategoryID", "CategoryName", "ListPrice", "StandardCost", "Weight", "WeightUom",
    "ProductStatus", "CreatedDate", "SellStartDate", "SellEndDate", "IsoCurrencyCode", "UpdatedDate", "CreatedBy", "UpdatedBy"
]
df = df.select([col(c).alias(tc) for c, tc in zip(df.columns, target_columns)])
# --- End column alignment ---

from pyspark.sql.types import IntegerType

df = df.withColumn("ProductID", col("ProductID").cast(IntegerType()))

# If you know the full schema, cast all columns accordingly:
# from pyspark.sql.types import StringType, DoubleType, ...
# df = df.withColumn("ProductID", col("ProductID").cast(StringType())) \
#        .withColumn("ListPrice", col("ListPrice").cast(DoubleType())) \
#        ... # repeat for all columns as needed

try:
    df.write \
      .format("delta") \
      .mode("append") \
      .option("overwriteSchema", "false") \
      .option("mergeSchema", "true") \
      .saveAsTable(TARGET_FULL_PATH)

    print(f"✅ Data loaded successfully to {TARGET_FULL_PATH}")

    # Verify the load
    result_count = spark.sql(f"SELECT COUNT(*) as count FROM {TARGET_FULL_PATH}").collect()[0]["count"]
    print(f"📊 Records in target table: {result_count}")

    # Show sample of loaded data
    print(f"\n📖 Sample from Silver table:")
    spark.sql(f"SELECT * FROM {TARGET_FULL_PATH} ORDER BY ProductID").show(10, truncate=False)

    print(f"🎉 Product data load complete!")

except Exception as e:
    print(f"❌ Error loading data to table: {str(e)}")
    raise